In [1]:
import pandas as pd
import numpy as np
import re
import time

import requests as rq
import bs4 as bs4
import tqdm
import glob
import json

# Coleta

In [2]:
df = pd.read_json("parsed_videos.json", lines=True)
df.head()

,link,query,titles
0,https://www.youtube.com/watch?v=4E0lIbH4WOU,machine-learning,Disciplina Machine Learning para Predições em ...
1,https://www.youtube.com/watch?v=EeMhj0sPrhE,machine-learning,Gradients are Not All You Need (Machine Learni...
2,https://www.youtube.com/watch?v=y6obVdOq93o,machine-learning,Análise de Dados - Concurso do TCU - Machine L...
3,https://www.youtube.com/watch?v=BRhz6v-jfMM,machine-learning,O que faz uma pessoa engenheira de Machine Lea...
4,https://www.youtube.com/watch?v=4jtV6nq_fFY,machine-learning,3D Print Failure Detection with Machine Learni...


In [31]:
lista_de_links = df['link']

# for link in lista_de_links:
#     response = rq.get(link)
#     print(link)
    
#     link_name = re.search("v=(.*)", link).group(1)
#     print('\n\n ', link_name)
    

#     with open("dados_brutos/video_{}.html".format(link_name), "w+") as output:
#         output.write(response.text)
    

    
lista_de_links.describe()

count                                             192
unique                                            103
top       https://www.youtube.com/watch?v=W5XNOmyJr6I
freq                                                2
Name: link, dtype: object

# Processamento dos dados

In [ ]:
with open("parsed_video_info.json", 'w+') as output:
    for video_file in tqdm.tqdm_notebook(sorted(glob.glob("./dados_brutos/video*"))):
        with open(video_file, 'r+') as inp:
            page_html = inp.read()
            parsed = bs4.BeautifulSoup(page_html, 'html.parser')

            class_watch = parsed.find_all(attrs={"class":re.compile(r"watch")})
            id_watch = parsed.find_all(attrs={"id":re.compile(r"watch")})
            channel = parsed.find_all("a", attrs={"href":re.compile(r"channel")})
            meta = parsed.find_all("meta")


            data = dict()

            for e in class_watch:
                colname = "_".join(e['class'])
                if "clearfix" in colname:
                    continue
                data[colname] = e.text.strip()

            for e in id_watch:
                colname = e['id']
                #if colname in output:
                #    print(colname)
                data[colname] = e.text.strip()

            for e in meta:
                colname = e.get('property')
                if colname is not None:
                    data[colname] = e['content']

            for link_num, e in enumerate(channel):
                data["channel_link_{}".format(link_num)] = e['href']


            output.write("{}\n".format(json.dumps(data)))


# Verificação

In [ ]:
df = pd.read_json("parsed_video_info.json", lines=True)
df.shape

In [ ]:
pd.set_option("display.max_columns", 166)
df.head(1)

In [ ]:
colunas_selecionadas = ['watch-title', 'watch-view-count', 'watch-time-text', 'content_watch-info-tag-list', 'watch7-headline',
                    'watch7-user-header', 'watch8-sentiment-actions', "og:image", 'og:image:width', 'og:image:height',
                    "og:description", "og:video:width", 'og:video:height', "og:video:tag", 'channel_link_0']

In [ ]:
df[colunas_selecionadas].head()

In [ ]:
df[colunas_selecionadas].to_feather("raw_data.feather")

In [ ]:
df[colunas_selecionadas].to_csv("raw_data_sem_labels.csv")